<a href="https://colab.research.google.com/github/tijeco/berteome/blob/dev/notebooks/scratch/04_augment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install "berteome @ git+https://github.com/tijeco/berteome@96c830219a42353732c2745916dd262f5f6b36f4"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/tijeco/berteome (to revision 96c830219a42353732c2745916dd262f5f6b36f4) to /tmp/pip-install-a2i2n4yr/berteome_4662e922da8844098ae7b69f1e444708
  Running command git clone -q https://github.com/tijeco/berteome /tmp/pip-install-a2i2n4yr/berteome_4662e922da8844098ae7b69f1e444708
  Running command git rev-parse -q --verify 'sha^96c830219a42353732c2745916dd262f5f6b36f4'
  Running command git fetch -q https://github.com/tijeco/berteome 96c830219a42353732c2745916dd262f5f6b36f4
  Running command git checkout -q 96c830219a42353732c2745916dd262f5f6b36f4
     |████████████████████████████████| 5.8 MB 10.0 MB/s 
     |████████████████████████████████| 571 kB 50.0 MB/s 
     |████████████████████████████████| 7.6 MB 42.3 MB/s 
     |████████████████████████████████| 182 kB 59.2 MB/s 
  Created wheel for berteome: filename=berteome-0.1.5-py3-none-any.whl size=12836 sha256=77

In [3]:
from berteome import berteome

In [4]:
berteome_models = berteome.modelLoader()
bert_tokenizer, bert_model = berteome_models.load_model("Rostlab/prot_bert")
mendel_berteome = berteome.modelPredDF("MENDEL",bert_tokenizer, bert_model)

Downloading:   0%|          | 0.00/81.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/361 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

Some weights of the model checkpoint at Rostlab/prot_bert were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Most probable residue

In [6]:
mendel_berteome.predDf.insert(3,"highestRes",mendel_berteome.predDf[list(mendel_berteome.aas)].idxmax(axis=1)) 

ValueError: ignored

In [7]:
mendel_berteome.predDf

,wt,wtIndex,wtScore,highestRes,A,C,D,E,F,G,...,M,N,P,Q,R,S,T,V,W,Y
0,M,1,0.076602,E,0.036697,0.011504,0.048245,0.118906,0.024072,0.039202,...,0.076602,0.072661,0.024722,0.038672,0.043105,0.070280,0.056544,0.049927,0.007781,0.021699
1,E,2,0.074830,L,0.045721,0.015662,0.041921,0.074830,0.037153,0.044325,...,0.043581,0.062667,0.025277,0.036911,0.055543,0.064425,0.049955,0.056789,0.012691,0.029893
2,N,3,0.041990,E,0.043564,0.009685,0.162590,0.184364,0.033782,0.044661,...,0.041484,0.041990,0.019992,0.025515,0.029433,0.048106,0.030303,0.054742,0.007430,0.024924
3,D,4,0.049748,L,0.042083,0.013244,0.049748,0.086194,0.039736,0.055911,...,0.040080,0.060822,0.032024,0.039689,0.046228,0.062323,0.044901,0.058937,0.010875,0.026596
4,E,5,0.086915,L,0.046641,0.018770,0.079822,0.086915,0.050638,0.050466,...,0.028962,0.062234,0.023879,0.030534,0.040489,0.065195,0.044938,0.068038,0.012156,0.038034
5,L,6,0.060736,E,0.038191,0.009217,0.065189,0.152547,0.020950,0.049525,...,0.040042,0.096484,0.020712,0.035022,0.046888,0.049071,0.046247,0.048276,0.010486,0.022727


# Top  mutational substitutions

Alright, so I think I have a pretty neat idea for this package, so I'm just going to document a few of my thoughts. This is an augment package, where the idea is to take a protein sequence and generate a bunch of point mutational variants. I think this would be quite handy in certain applications, for instance antimicrobial peptide classification. There aren't too many known examples of antimicrobial peptides, but tons and tons of examples of non-antimicrobial peptides. If this were a computer vision problem, say we were trying to build a classifier for four leaf clovers vs three leaf clovers, you'd have a much easier time finding examples of three leaf vs four leaf. To compensate for this so that you have an evenish sampling you can augment the dataset byt manipulating the pictures you do have and jittering the images about to get kind of pseudoreplicates. Sure, there's lot's of ways to approach that problem, but for protein sequences it is not so easy to augment those sequences. If they are all in the same gene family, you can make a multiple sequence alignment of them and reconstruct their ancestral states using phylogenetic approaches, this would double the data size! But if that isn't the case then what can you do? Well you can take the single point mutation approach. Say for a protein of length L, generate N variants where you randomly mutate a single amino residue. That is sort of fine in practice, but we know that some mutations are more feleterious than others, if only there were a way to determine which mutations weren't likely to be deleterious... Aha! So I think we can take the dataframes generated from ESM or prot_bert and generate the top k most likely mutational variants per residue. Let's think about how that could potentially augment the data set. So for a single peptide of length L, if we take the top k substitutions per residue, that gives L*k variants, for the toy MENDEL example where L is 6, this would generate 30 variants! Which is quite substantial, for larger proteins I think you could get quite a reasanobly large number of variants.

Alright, so here is what I plan to do, the prediction dataframe is pretty core to this, though for demonstration purposes, I'll probably just handmake a MENDEL dataframe. From that the main function will be an augmentPep function which will take a prediction df from a single peptide, as well k to dictate how many mutational variants per residue. An obvious caveat is the fact that the wild type residue will probably often be in the top k possible substitutions, so there will need to be a small check to insure we are getting the top k residues that are not wt. Beyond that, two aditional functions come to mind than can be built on top of this, augmentFasta and augmentPeps, which just takes augmentPep and applies it to either a fasta file or an iterable list of peptides. I should also think about how to name mutants, I feel like just posResidue + ResidueSub would  be sufficient to tag that to the original id.

Let's try to jump in this, and make a dummy MENDEL dataframe.

In [ ]:
import pandas as pd

In [ ]:
mendelDF = pd.DataFrame.from_dict({
	#	 A,C,D,E,F,G,H,I,K,L,M,N,P,Q,R,S,T,V,W,Y
	"M":[0,0,0,0,0,0,0,0,0,8,9,7,0,0,0,0,0,0,0,0],
	"E":[0,0,7,8,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
	"N":[0,0,0,0,0,0,0,0,0,9,8,7,0,0,0,0,0,0,0,0],
	"D":[0,7,8,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
	# "E":[0,0,9,8,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0], # redundant... for the example
	"L":[0,0,0,0,0,0,0,0,8,9,7,0,0,0,0,0,0,0,0,0]
}, orient="index", columns=list("ACDEFGHIKLMNPQRSTVWY")).reset_index().rename(columns={"index": "wt"})


mendelDF

,wt,A,C,D,E,F,G,H,I,K,...,M,N,P,Q,R,S,T,V,W,Y
0,M,0,0,0,0,0,0,0,0,0,...,9,7,0,0,0,0,0,0,0,0
1,E,0,0,7,8,9,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,N,0,0,0,0,0,0,0,0,0,...,8,7,0,0,0,0,0,0,0,0
3,D,0,7,8,9,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,L,0,0,0,0,0,0,0,0,8,...,7,0,0,0,0,0,0,0,0,0


Alright so this very much a barebones dummy df, but I think it will work.

So for this example I would only want to sample k=2, to go through each residue it should be something like

1. M: L,N
2. E: D,F
3. N: L,M
4. D: C,E
<!-- 5. E: D,F -->
6. L: K,M

So that should give us 12 variant sequences! (Edit:10..)

So let me go ahead and think about what the logic needs to be for this. There is certainly a chance that I will be able to use some fancy pandas stuff to do this, but for now I will just focus on what might work.

So I definitely know that I need to grab the top k values in each column.. wait that's not right. No, I need to get the top values for each row! Then the tricky part is figuring out if the column name is the same value as the index.


Oh gravy, there definitely has to be some sort of SQLy or panday way to get this


In [ ]:
def augmentPep(df, k):
	seqList = list(df["wt"])
	variantDict = {}
	for index, row in df.iterrows():
		scores = row[list("ACDEFGHIKLMNPQRSTVWY")]
		top_k_scores = scores.where(scores.index != row["wt"]).sort_values(ascending=False).head(k)

		top_k_subs = list(top_k_scores.index)
		for res in top_k_subs:
			
			seqCopy = seqList.copy()
			seqCopy[index] = res
			variantDict[f"{index}x{res}"] = ''.join(seqCopy) 

	return variantDict


{'0xL': 'LENDL',
 '0xN': 'NENDL',
 '1xF': 'MFNDL',
 '1xD': 'MDNDL',
 '2xL': 'MELDL',
 '2xM': 'MEMDL',
 '3xE': 'MENEL',
 '3xC': 'MENCL',
 '4xK': 'MENDK',
 '4xM': 'MENDM'}

In [ ]:
assert augmentPep(mendelDF, 2) == {'0xL': 'LENDL',
 '0xN': 'NENDL',
 '1xF': 'MFNDL',
 '1xD': 'MDNDL',
 '2xL': 'MELDL',
 '2xM': 'MEMDL',
 '3xE': 'MENEL',
 '3xC': 'MENCL',
 '4xK': 'MENDK',
 '4xM': 'MENDM'}

ALright!! In principle, that's working!!

The output structure needs to be modified so that the mutational variant is annotated as "pos_res".

I think I like this..

Let's see how it works on esm/prot_bert dataframes

In [ ]:
from berteome import prot_bert

Some weights of the model checkpoint at Rostlab/prot_bert were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
mendel_prot_bert_DF = prot_bert.bertPredictionDF("MENDEL")

In [ ]:
mendel_prot_bert_DF

,wt,wtIndex,A,C,D,E,F,G,H,I,...,M,N,P,Q,R,S,T,V,W,Y
0,M,1,0.036685,0.011501,0.048229,0.118868,0.024064,0.039190,0.012617,0.066477,...,0.076580,0.072637,0.024714,0.038660,0.043091,0.070257,0.056526,0.049911,0.007779,0.021692
1,E,2,0.045712,0.015659,0.041913,0.074816,0.037146,0.044317,0.018260,0.073063,...,0.043572,0.062655,0.025272,0.036905,0.055532,0.064412,0.049945,0.056779,0.012689,0.029887
2,N,3,0.043558,0.009684,0.162566,0.184336,0.033777,0.044654,0.012353,0.052622,...,0.041478,0.041984,0.019989,0.025511,0.029428,0.048098,0.030299,0.054734,0.007428,0.024920
3,D,4,0.042079,0.013243,0.049744,0.086189,0.039733,0.055907,0.016860,0.073291,...,0.040078,0.060817,0.032022,0.039686,0.046224,0.062319,0.044898,0.058933,0.010875,0.026594
4,E,5,0.046638,0.018769,0.079816,0.086908,0.050634,0.050462,0.022395,0.074495,...,0.028960,0.062229,0.023877,0.030532,0.040486,0.065190,0.044934,0.068032,0.012155,0.038031
5,L,6,0.035695,0.008615,0.060928,0.142576,0.019581,0.046287,0.013043,0.060374,...,0.037424,0.090177,0.019358,0.032733,0.043823,0.045863,0.043224,0.045121,0.009800,0.021241


In [ ]:
augmentPep(mendel_prot_bert_DF, 2)

{'0xE': 'EENDEL',
 '0xK': 'KENDEL',
 '1xL': 'MLNDEL',
 '1xK': 'MKNDEL',
 '2xE': 'MEEDEL',
 '2xD': 'MEDDEL',
 '3xL': 'MENLEL',
 '3xK': 'MENKEL',
 '4xL': 'MENDLL',
 '4xD': 'MENDDL',
 '5xE': 'MENDEE',
 '5xK': 'MENDEK'}

In [ ]:
from berteome import esm

In [ ]:
mendel_esm_DF = esm.esmPredictionDF("MENDEL")

In [ ]:
augmentPep(mendel_esm_DF, 2)

{'0xE': 'EENDEL',
 '0xD': 'DENDEL',
 '1xL': 'MLNDEL',
 '1xS': 'MSNDEL',
 '2xE': 'MEEDEL',
 '2xL': 'MELDEL',
 '3xL': 'MENLEL',
 '3xK': 'MENKEL',
 '4xL': 'MENDLL',
 '4xS': 'MENDSL',
 '5xE': 'MENDEE',
 '5xK': 'MENDEK'}

As far as I'm concerned, this is working! It shouldn't be too difficult to use this to make a `augmentPeps` and `augmentFasta`, which takes multiple peptides and returns their top k variants.

I don't entirely need this right this moment, so maybe I should just work on that when I definitely need it or at least have more time, because it will probably take a couple of hours to finish.